In [ ]:
import cv2
import numpy as np

In [ ]:
#initialise

MHI_DURATION = 36 #number of concurrent frames in MHI
time = 0

# Flow Options:
alpha = 0.012
ratio = 0.75
minWidth = 20
nOuterFPIterations = 7
nInnerFPIterations = 1
nSORIterations = 30
colType = 0  # 0 or default:RGB, 1:GRAY (but pass gray image with shape (h,w,1))

In [ ]:
#OF-MHI Brox

#input
cap = cv2.VideoCapture('C:/Users/Aditya/Downloads/JESTER DATASET/20bn-jester-v1/1/%5d.jpg')
fgbg = cv2.cuda.createBackgroundSubtractorMOG2(detectShadows = False)

ret, frame = cap.read()
h, w = frame.shape[:2]
motion_history = np.zeros((h, w), np.float32)
motion_history2 = np.zeros((h, w), np.float32)
prev = frame.copy()


#output video to directory
res = (w, h) #resolution
fourcc = cv2.VideoWriter_fourcc(*'MJPG') #codec
name = '/content/drive/MyDrive/RSIP_Project/output.avi'
out = cv2.VideoWriter(name, fourcc, 20.0, res)


#loop remaining frames
while(1):

    #for i in range(3):
    ret, frame = cap.read()
    
    gpu_frame = cv2.cuda_GpuMat()
    gpu_frame.upload(frame)

    if not ret:
        cap.release()
        break

    fgmask = fgbg.apply(gpu_frame, 0.1)
    kernel = np.ones((2, 2), np.uint8)
    erode = cv2.erode(fgmask, kernel, iterations = 2)

    #brox optical flow
    im1 = prev.astype(float) / 255.
    im2 = frame.astype(float) / 255.

    u, v, im2W = pyflow.coarse2fine_flow(im1, im2, alpha, ratio, minWidth, nOuterFPIterations, nInnerFPIterations, nSORIterations, colType)
    flow = np.concatenate((u[..., None], v[..., None]), axis=2)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[...,0] = ang * 180 / np.pi / 2
    hsv[...,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    hsv[...,1] = 255
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    opmag = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    ret, thresh = cv2.threshold(opmag, 0, 255, cv2.THRESH_TOZERO)

    cv2_imshow(thresh)

    kernel2 = np.ones((2, 2), np.uint8)
    erode2 = cv2.erode(thresh, kernel2, iterations = 1)

    prev = frame

    #merge images
    merged = cv2.addWeighted(erode, 0.5, erode2, 0.5, 0)

    # update motion history
    time += 1
    cv2.motempl.updateMotionHistory(erode, motion_history, time, MHI_DURATION)
    mhi = np.uint8(np.clip((motion_history - (time - MHI_DURATION)) / MHI_DURATION, 0, 1)*255)

    cv2.motempl.updateMotionHistory(erode2, motion_history2, time, MHI_DURATION)
    mhi2 = np.uint8(np.clip((motion_history2 - (time - MHI_DURATION)) / MHI_DURATION, 0, 1)*255)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    mhi = np.uint8(mhi)

    out.write(mhi)
    #cv2_imshow(opmag)
    
    if 0xFF & cv2.waitKey(30) == 27:
        break

out.release()
cv2_imshow(mhi)
cv2_imshow(mhi2)

ret, thresh1 = cv2.threshold(mhi, 127, 255, cv2.THRESH_BINARY)
ret, thresh2 = cv2.threshold(mhi2, 127, 255, cv2.THRESH_BINARY)

mask = cv2.bitwise_xor(thresh1, thresh2)
cv2_imshow(mask)

final = cv2.add(mask, mhi)
cv2_imshow(final)

cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.5.2) :-1: error: (-5:Bad argument) in function 'apply'
> Overload resolution failed:
>  - cuda_BackgroundSubtractorMOG2.apply() missing required argument 'stream' (pos 3)
>  - cuda_BackgroundSubtractorMOG2.apply() missing required argument 'stream' (pos 3)
>  - cuda_BackgroundSubtractorMOG2.apply() missing required argument 'stream' (pos 3)


In [ ]:
import cv2
import numpy as np

#fresh start
print(cv2.cuda.getCudaEnabledDeviceCount())

vidPath = 'C:/Users/Aditya/Downloads/JESTER DATASET/20bn-jester-v1/10/%5d.jpg'
lr = 0.05
check_res = False
frame_device = cv2.cuda_GpuMat()

#taking input, pass 0 to use webcam data
cap = cv2.VideoCapture(vidPath)

bgmog2_device = cv2.cuda.createBackgroundSubtractorMOG2()

brox_of = cv2.cuda_BroxOpticalFlow.create(0.2, 50.0, 0.8, 5, 150, 10) 
#(alpha ,gamma ,scale ,inner_iterations ,outer_iterations ,solver_iterations)
TVL1_of = cv2.cuda_OpticalFlowDual_TVL1.create(0.25, 0.15, 0.3, 5, 5, 0.01, 300, 0.8, 0.0, False)
#(double tau, double lambda, double theta, int nscales, int warps, double epsilon, int iterations, double scaleStep, double gamma, bool useInitialFlow)

def bgmogCuda(frame,lr,store_res=False):
    frame_device.upload(frame)
    fg_device = bgmog2_device.apply(frame_device,lr,cv2.cuda.Stream_Null())
    fg_host = fg_device.download()
    if(store_res):
        gpu_res.append(np.copy(fg_host))
    return fg_host
        
gpu_res = []

gpu_prev = cv2.cuda_GpuMat()
gpu_frame = cv2.cuda_GpuMat()

ret, frame = cap.read()
frame = np.float32(frame)/255.0
gpu_prev.upload(frame)
gpu_prev = cv2.cuda.cvtColor(gpu_prev, cv2.COLOR_BGR2GRAY)
print("Type prev = ",gpu_prev.type())

h, w = frame.shape[:2]
temp = np.zeros((h, w), np.float32)
temp = gpu_prev.download()

cv2.imshow('temp', temp)
gpu_v = cv2.cuda_GpuMat(gpu_prev.size(), cv2.CV_32FC1)

#nv_of = cv2.cuda_NvidiaOpticalFlow_2_0.create(gpu_prev.size())

#loop frames
while(1):

    ret, frame = cap.read()
    
    if not ret:
        cap.release()
        break
        
    #bgsgm
    fgmask = bgmogCuda(frame, lr, store_res=check_res)
    kernel = np.ones((3, 3), np.uint8)
    erode = cv2.erode(fgmask, kernel, iterations = 1)
    cv2.imshow('bgmog', erode)
    
    #brox optical flow opencv cuda
    frame = np.float32(frame)/255.0
    
    gpu_frame.upload(frame)
    
    #had to add this because of some bug in library
    if(gpu_prev.type() != 5):
        gpu_prev = cv2.cuda.cvtColor(gpu_prev, cv2.COLOR_BGR2GRAY)
        
    gpu_frame = cv2.cuda.cvtColor(gpu_frame, cv2.COLOR_BGR2GRAY)

    gpu_flow = brox_of.calc(gpu_prev, gpu_frame, None) #FLOW COMPUTATION
    
    gpu_flow_x = cv2.cuda_GpuMat(gpu_flow.size(), cv2.CV_32FC1)
    gpu_flow_y = cv2.cuda_GpuMat(gpu_flow.size(), cv2.CV_32FC1)
    cv2.cuda.split(gpu_flow, [gpu_flow_x, gpu_flow_y], cv2.cuda.Stream_Null())
    
    gpu_magnitude, gpu_angle = cv2.cuda.cartToPolar(gpu_flow_x, gpu_flow_y, angleInDegrees=False)
    
    # set value to normalized magnitude from 0 to 1
    gpu_v = cv2.cuda.normalize(gpu_magnitude, 0.0, 1.0, cv2.NORM_MINMAX, -1)

    res = gpu_v.download()

    cv2.imshow("result", res)    
    cv2.imshow("original", frame)

    gpu_prev.upload(frame)

    if 0xFF & cv2.waitKey(1) == 27:
        break
        
cv2.waitKey(0)
cv2.destroyAllWindows()

1
Type prev =  5


In [ ]:
#MHI giving bad results so only OF for now

import cv2
import numpy as np
import pandas as pd
import os
from tifffile import imread, imwrite

#148085

vidPath = 'C:/Users/Aditya/Downloads/JESTER DATASET/20bn-jester-v1/1/%5d.jpg'

cap = cv2.VideoCapture(vidPath) #give 0 for webcam data

brox_of = cv2.cuda_BroxOpticalFlow.create(0.2, 50.0, 0.75, 5, 150, 10) 
#(alpha ,gamma ,scale ,inner_iterations ,outer_iterations ,solver_iterations)

gpu_prev = cv2.cuda_GpuMat()
gpu_frame = cv2.cuda_GpuMat()

ret, frame = cap.read()
frame = np.float32(frame)/255.0
gpu_prev.upload(frame)
gpu_prev = cv2.cuda.cvtColor(gpu_prev, cv2.COLOR_BGR2GRAY)

d = 1
#out = np.zeros(frame.shape[:2], np.float32)

#loop frames
while(1):
    ret, frame = cap.read()
    
    if not ret:
        cap.release()
        break
        
    #brox optical flow opencv cuda
    frame = np.float32(frame)/255.0
    gpu_frame.upload(frame)
    
    #had to add this because of some bug in library
    if(gpu_prev.type() != 5):
        gpu_prev = cv2.cuda.cvtColor(gpu_prev, cv2.COLOR_BGR2GRAY)
        
    #change rgb to b/w
    gpu_frame = cv2.cuda.cvtColor(gpu_frame, cv2.COLOR_BGR2GRAY)
    
    #FLOW COMPUTATION
    gpu_flow = brox_of.calc(gpu_prev, gpu_frame, None)
    
    gpu_flow_x = cv2.cuda_GpuMat(gpu_flow.size(), cv2.CV_32FC1)
    gpu_flow_y = cv2.cuda_GpuMat(gpu_flow.size(), cv2.CV_32FC1)
    cv2.cuda.split(gpu_flow, [gpu_flow_x, gpu_flow_y], cv2.cuda.Stream_Null())
    
    gpu_magnitude, gpu_angle = cv2.cuda.cartToPolar(gpu_flow_x, gpu_flow_y, angleInDegrees=False)
    
    # set value to normalized magnitude from 0 to 1
    gpu_magnitude = cv2.cuda.normalize(gpu_magnitude, 0, 255, cv2.NORM_MINMAX, -1)
    
    mag2d = gpu_magnitude.download() 
    #mag2d = cv2.erode(mag2d, (11, 11), iterations = 3) #was testing erosion
    mag3d = np.expand_dims(mag2d, axis=2)
    #print(mag3d.shape)
    mag3d = np.asarray(np.moveaxis(mag3d, -1, 0))
    #print(mag3d.shape)
    
    if(d == 1):
        out = mag3d.copy()
        #print(out.shape)
    else:
        out = np.concatenate((out, mag3d), axis=0)
    #os.chdir('C:\\Users\\Aditya\\Downloads\\JESTER DATASET\\output\\1')
    filename = f"C:/Users/Aditya/Downloads/JESTER DATASET/output/{d:05d}.jpg"
    #cv2.imwrite(filename, mag2d)
    d += 1    
    gpu_prev.upload(frame)
    
    if 0xFF & cv2.waitKey(1) == 27:
        break

#print(out.shape)
filename = f"C:/Users/Aditya/Downloads/JESTER DATASET/output/{d:02d}.tiff"
imwrite(filename, out, photometric='minisblack')
temp = imread(filename)
print(temp.shape)

cv2.waitKey(0)
cv2.destroyAllWindows()

(36, 100, 176)


In [ ]:
print(out.shape)

(100, 176, 36)


In [ ]:
os.chdir('C:\\Users\\Aditya\\Downloads\\JESTER DATASET')
os.getcwd()
#C:\Users\Aditya\Downloads\JESTER DATASET\output\1

'C:\\Users\\Aditya\\Downloads\\JESTER DATASET'

In [ ]:
#trying MFFs

#method - count number of frames in folder
#skip first 4 frames, then divide remaining frames by 8. call the quotient n
#every nth frame after leaving the first 4 frames has to be the rgb for the segment
#store OFs of past 3 frames at any given time in 3 arrays
#write the image as .TIFF or .npy
#profit?

import cv2
import numpy as np
import pandas as pd
import os
from tifffile import imread, imwrite

#148085 #looks cool

vidPath = 'C:/Users/Aditya/Downloads/JESTER DATASET/20bn-jester-v1/1/%5d.jpg'
folder = 'C:/Users/Aditya/Downloads/JESTER DATASET/20bn-jester-v1/1/'

#interval between rgb frames
n = len(os.listdir(folder))
n = n - 4
n = n // 8

cap = cv2.VideoCapture(vidPath) #give 0 for webcam data

brox_of = cv2.cuda_BroxOpticalFlow.create(0.2, 50.0, 0.75, 5, 150, 10) 
#(alpha ,gamma ,scale ,inner_iterations ,outer_iterations ,solver_iterations)

#creating GPUMATS
gpu_prev = cv2.cuda_GpuMat()
gpu_frame = cv2.cuda_GpuMat()

#taking first frame
ret, frame = cap.read()
frame = np.float32(frame)/255.0
gpu_prev.upload(frame)
gpu_prev = cv2.cuda.cvtColor(gpu_prev, cv2.COLOR_BGR2GRAY)

#frame counter
d = 1

#segment frame counter
s = 1

#segment counter
k = 1

#loop frames
while(1):
    ret, frame = cap.read()
    
    if not ret:
        cap.release()
        break
    
    #brox optical flow opencv cuda
    frame = np.float32(frame)/255.0
    
    gpu_frame.upload(frame)
    
    #had to add this because of some bug in library
    if(gpu_prev.type() != 5):
        gpu_prev = cv2.cuda.cvtColor(gpu_prev, cv2.COLOR_BGR2GRAY)
        
    #change rgb to b/w
    gpu_frame = cv2.cuda.cvtColor(gpu_frame, cv2.COLOR_BGR2GRAY)
    
    #FLOW COMPUTATION
    gpu_flow = brox_of.calc(gpu_prev, gpu_frame, None) #2 vectors
    
    #splitting flow into 2
    gpu_flow_x = cv2.cuda_GpuMat(gpu_flow.size(), cv2.CV_32FC1)
    gpu_flow_y = cv2.cuda_GpuMat(gpu_flow.size(), cv2.CV_32FC1)
    cv2.cuda.split(gpu_flow, [gpu_flow_x, gpu_flow_y], cv2.cuda.Stream_Null())
    
    #storing mag and and of the flow field
    gpu_magnitude, gpu_angle = cv2.cuda.cartToPolar(gpu_flow_x, gpu_flow_y, angleInDegrees=False)
    
    # set value to normalized magnitude from 0 to 255
    gpu_magnitude = cv2.cuda.normalize(gpu_magnitude, 0, 255, cv2.NORM_MINMAX, -1)
    
    temp = gpu_flow_x.download()
    cv2.imshow('temp', temp)
    
    mag2d = gpu_magnitude.download() 
    #mag2d = cv2.erode(mag2d, (11, 11), iterations = 3) #was testing erosion
    mag3d = np.expand_dims(mag2d, axis=2)
    #print(mag3d.shape)
    #TIFF writing has some bug so had to do this
    #mag3d = np.asarray(np.moveaxis(mag3d, -1, 0)) #uncomment this if dimension probs later
    
    if(s == n):
        rgb = frame.copy()
        out = np.concatenate((rgb, of3, of2, of1), axis=2)
        #out = np.concatenate((rgb, of3, of2, of1), axis=0) #uncomment this if dim probs later.
        filename = f"C:/Users/Joy/Downloads/JESTER DATASET/output/MFFtryTIFF/{k}.tif"
        filepath = f"C:/Users/Joy/Downloads/JESTER DATASET/output/MFFtryNPY/{k}"
        imwrite(filename, out)
        print(out.shape)
        np.save(filepath, out, allow_pickle=True)
        k += 1
        s = 0
    
    if k == 9:
        break
    if d>2:
        of1 = of2.copy()
    if d>1:
        of2 = of3.copy()
    of3 = mag3d.copy()
    
    d += 1
    s += 1  
    gpu_prev.upload(frame)
    
    if 0xFF & cv2.waitKey(1) == 27:
        break
    
cv2.waitKey(0)
cv2.destroyAllWindows()

(100, 176, 6)
(100, 176, 6)
(100, 176, 6)
(100, 176, 6)
(100, 176, 6)
(100, 176, 6)
(100, 176, 6)
(100, 176, 6)


In [ ]:
arr = np.load("C:/Users/Aditya/Downloads/JESTER DATASET/output/1.npy", allow_pickle=True)
print(arr.shape)

(100, 176, 6)
